# 1 - Importar pandas para ler os arquivos CSV e transformar em data frames

In [ ]:
# Import pandas para ler o arquivo CSV
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

display(train_df.head())

# 2 - Ver tipos de dados e Valores Ausentes

In [ ]:
# Ver tipos de dados
display(train_df.info())

# Ver valores ausentes
display(train_df.isnull().sum())

# 3 - Tratar Valores Ausentes

In [ ]:
# Adicionar a linhas com a coluna "Age" faltando com o valor da mediana
train_df["Age"].fillna(train_df["Age"].median(), inplace=True)

# Remover a coluna "Cabin", pois ha mais de 70% das linhas com a coluna ausente
if "Cabin" in train_df.columns:
    train_df.drop(columns="Cabin", inplace=True)

# Preencher "Embarked" com o valor mais comum (moda)
train_df["Embarked"].fillna(train_df["Embarked"].mode()[0], inplace=True)

# 4 - Transformar Variáveis Categóricas

In [ ]:
# Codificando a Coluna "Sex"
train_df["Sex"] = train_df["Sex"].map({'male': 0, 'female':1})

# Codificar "Embarked" em 3 novas colunas
embarked_dummies = pd.get_dummies(train_df["Embarked"], prefix="Embarked")
train_df = pd.concat([train_df, embarked_dummies], axis=1)

train_df.drop(columns="Embarked", inplace=True)


In [ ]:
train_df[['Embarked_C', 'Embarked_Q', 'Embarked_S']] = train_df[['Embarked_C', 'Embarked_Q', 'Embarked_S']].astype(int)

display(train_df.head())

# 5 - Separar dados para treinamento e teste do modelo

In [ ]:
from sklearn.model_selection import train_test_split

# Variáel a ser prevista
y = train_df["Survived"]

# Entradas (Tirando colunas que não iriam auxiliar o modelo a prever)
X = train_df.drop(columns=['Survived', 'PassengerId', 'Name', 'Ticket'])

# dividindo o dataframe pra treinamento e teste

X_train, X_test, y_train, y_test = train_test_split(
    X,y, test_size=0.2, random_state=42
)

# 6 - Treino e Teste do modelo (RandomForestClassifier)

In [ ]:
# Import do modelo
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Criação e treinamento do modelo
modelo = RandomForestClassifier(random_state=42)
modelo.fit(X_train, y_train)

# Teste
y_pred = modelo.predict(X_test)

print("Acuracia:", accuracy_score(y_test, y_pred))
print("Matriz Confusão:")
print(confusion_matrix(y_test, y_pred))
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))

# 7 - Visualização da importância das fetures no Random Forest

In [ ]:
import matplotlib.pyplot as plt

# importancia das features
importancias = modelo.feature_importances_

# nome das colunas usadas no modelo
nomes_colunas = X.columns

# Criação de dataframe para visualização
df_importancia = pd.DataFrame({
    'Feature': nomes_colunas,
    'Importância': importancias,
}).sort_values(by='Importância', ascending=False)

# Exibe como tabela
display(df_importancia)

# Exibe Gráfico
plt.figure(figsize=(10,6))
plt.barh(df_importancia['Feature'], df_importancia['Importância'])
plt.xlabel('Importância')
plt.title('Importância das Features - Random Forest')
plt.gca().invert_yaxis()  #Coloca as mais importantes no topo
plt.show()

* Teste para tabela sem as colunas 'Sex' e 'Embarked'

In [ ]:
# Remove features de baixa importância
X_reduzido = X.drop(columns=['Sex', 'Embarked_C', 'Embarked_Q', 'Embarked_S'])

# Divide de novo
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_reduzido, y, test_size=0.2, random_state=42
)

# Treina novo modelo
modelo_reduzido = RandomForestClassifier(random_state=42)
modelo_reduzido.fit(X_train_r, y_train_r)

# Avalia
y_pred_r = modelo_reduzido.predict(X_test_r)

from sklearn.metrics import accuracy_score, classification_report
print("Acurácia (sem Sex e Embarked):", accuracy_score(y_test_r, y_pred_r))
print(classification_report(y_test_r, y_pred_r))


Aqui observa-se uma leve queda na acurácia do modelo. Apesar da aparente baixa importância dessas colunas o modelo reagiu mal com as alterações.